In [1]:
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import os
from mutual_information import *
import mutual_information as mtin

In [42]:
# mtin.default_fn_equ_params["circular_stim"] = True
# mtin.default_fn_equ_params["stim"] = [[[25, 40], [55, 55], 10]]
T = 20_000
dt = 0.01
D = 1
mtin.default_fn_equ_params.update({
    'dt': dt,
    'T': T,
    'D': D,
    'stim': [[[250, 350], [45, 65], [45, 65]]],
})
mtin.default_fn_equ = FHN(**default_fn_equ_params)
# mtin.default_fn_equ = DiffusionPDE(D)
mtin.solve_default_fn_equ(resolve=True)
# grid, vw0 = mtin.get_default_fn_initial_state()
# vw0[0].insert((50, 50), 10)
# memory_storage = MemoryStorage()
# mtin.default_fn_equ.solve(vw0[0], t_range=T * dt, dt=dt, tracker=[memory_storage.tracker(dt)])
# mtin.default_fn_sol = np.array(memory_storage.data)



In [43]:
vs = ws = mtin.default_fn_sol
params = mtin.default_fn_equ_params

In [44]:
vs = mtin.default_fn_sol[:, 0]
ws = mtin.default_fn_sol[:, 1]
params = mtin.default_fn_equ_params

## Animation

In [45]:
vmin = None
vmax = None

In [46]:
vmin = -2
vmax = 2

In [47]:
if False:
    params: dict
    dir = "./fn_images/fixed_z_foolery/"
    os.makedirs(dir, exist_ok=True)
    save_params(params, f"{dir}/params.json")
    for a in tqdm(range(0, 1000, 1)):
        fig = plt.figure()
        fig.set_size_inches((8, 3))
        axs = fig.subplots(1, 2, squeeze=False)
        axes_image = axs[0, 0].imshow(vs[a], vmin=vmin, vmax=vmax)
        plt.colorbar(axes_image, ax=axs[0, 0])
        axes_image = axs[0, 1].imshow(ws[a], vmin=vmin, vmax=vmax)
        plt.colorbar(axes_image, ax=axs[0, 1])
        plt.savefig(f"{dir}/{a}.png")
        plt.close(fig)

In [48]:
from animate import animate_video
dir = "./fn_videos/fn_dt_0.01/"
os.makedirs(dir, exist_ok=True)
save_params(params, f"{dir}/params.json")
# vws = np.transpose(np.concatenate((vs, np.zeros(gap_shape), ws), axis=2), (0, 2, 1))
animate_video(f"{dir}/vs.mp4", vs)
animate_video(f"{dir}/ws.mp4", ws)



nt = 20001, nx = 100, ny = 100
i = 20000/20001
nt = 20001, nx = 100, ny = 100
i = 20000/20001
